In [12]:
from unicodedata import decimal
import clr
import sys
import builtins
import quotecomPy
from quotecomPy import quoteCom
from Intelligence import IdxKind 

# 設定DLL的路徑
dll_path = os.path.abspath("QuoteComExamplePy")
sys.path.append(dll_path)

# 引用必要的DLL
clr.AddReference("Package")
clr.AddReference("PushClient")
clr.AddReference("QuoteCom")

# 引用模組

quotecomPy.initialize()

# 連線及登入函式
def connect_to_quote(uid, pwd):
    host = 'iquotetest.kgi.com.tw'
    port = 8000
    quoteCom.Connect2Quote(host, port, uid, pwd, ' ', '')
    quoteCom.OnRcvMessage += on_quote_rcv_message
    quoteCom.OnGetStatus += on_quote_get_status

def on_quote_get_status(sender, status, msg):
    if status == COM_STATUS.CONNECT_READY:
        print(f"STATUS: CONNECT_READY: [{msg}]")
    elif status == COM_STATUS.LOGIN_READY:
        print(f"STATUS: LOGIN_READY: [{msg}]")
    elif status == COM_STATUS.LOGIN_FAIL:
        print(f"STATUS: LOGIN_FAIL: [{msg}]")
    # 更多狀態處理

# 註冊即時報價函式
def register_real_time_quotes():
    match_ID_info = quoteCom.SubQuotesMatch('2330')  # 註冊多個商品
    return match_ID_info

# 解除註冊即時報價函式
def unregister_real_time_quotes():
    quoteCom.UnSubQuotesMatch('2883|2330|0050')  # 解除註冊多個商品

# 資料接收事件
def on_quote_rcv_message(sender, pkg):
    if pkg.DT == DT.QUOTE_STOCK_MATCH1:
        print(f"成交商品: {pkg.StockNo}")
        print(f"成交價: [{pkg.Match_Price}]")
        print(f"成交量: [{pkg.Match_Qty}]")

quoteCom.OnRcvMessage += on_quote_rcv_message
quoteCom.OnGetStatus += on_quote_get_status

# 查詢最新行情報價
def retrieve_last_price_stock():
    quoteCom.RetriveLastPriceStock('2883')

def on_quote_rcv_message(sender, pkg):
    if pkg.DT == DT.QUOTE_LAST_PRICE_STOCK:
        print(f"商品: {pkg.StockNo}")
        print(f"成交價: [{pkg.LastMatchPrice}]")
        print(f"成交量: [{pkg.LastMatchQty}]")
        for i in range(5):
            print(f"第[{i+1}]檔--委買[價: {pkg.BUY_DEPTH[i].PRICE} 量: {pkg.BUY_DEPTH[i].QUANTITY}]  委賣[價: {pkg.SELL_DEPTH[i].PRICE} 量: {pkg.SELL_DEPTH[i].QUANTITY}]")


QuoteCom API initialize........


In [13]:
register_real_time_quotes()

-1

In [5]:
import requests
import numpy as np
import time
import datetime
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from random import randint
import pandas as pd
import openpyxl
from tqdm import trange
import yfinance as yf
from stock_reminder_package.stock_reminder_utils import *
sys.path.append('C:/Users/User/Desktop/StockInfoHub')
from Shared_Modules.shared_functions import *
from Shared_Modules.shared_variables import *
import warnings
warnings.filterwarnings('ignore')
# 字體顏色
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR
# LINE Notify 權杖
token = 'F7cbo3hpi6SKYA8KCXXERUSOroaJTZOyO9QaCetMDOU'
notify_ornot = True # True : 發送訊息
program_test = False # True : 測試模式
program_start = False # True : 不用等開盤
# 假日不執行
n_day_ago = -0
day = datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=n_day_ago)) , '%Y-%m-%d' )
if str(day).split(' ')[0] in HOLIDAY:
    print(f'Today is holiday : {day}')
    line_notify(f'{day}放假不執行stocknotify.py', TOKEN_FOR_NOTIFY, notify_ornot)
    sys.exit()

In [6]:
johnny_allmoney = 1800000
jack_allmoney = 100000
# 曝險大小，同時持有總資金幾%部位
risk = 30
# 最大部位總資金幾%
maxposition = 17 
johnny_maxposition = johnny_allmoney * maxposition / 100
jack_maxposition = jack_allmoney * maxposition / 100
# 自動下載ChromeDriver
# print(ChromeDriverManager().install())
chrome_path = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
# service = ChromeService(executable_path=ChromeDriverManager(path=chrome_path).install())
service = ChromeService(executable_path=ChromeDriverManager().install())
# 關閉通知提醒
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
# 開啟瀏覽器
driver = webdriver.Chrome(service=service, options=chrome_options)

In [10]:
# 定義檢查 Breakout 的函數
def check_breakout(current_price, last_price, breakout_prices, breakout_notes):
    alerts = ['突破買']
    if breakout_prices is not None and breakout_notes is not None:
        for price, note in zip(breakout_prices, breakout_notes):
            if last_price <= price <= current_price:
                price_diff_percentage = (current_price - price) / price
                alerts.append(f'Price: {price}📈\nNote: {note}\nPrice Diff: {price_diff_percentage:.2%}\n')
    return alerts

# 定義檢查 Profit 的函數
def check_profit(current_price, last_price, profit_prices, profit_notes):
    alerts = ['停利']
    if profit_prices is not None and profit_notes is not None:
        for price, note in zip(profit_prices, profit_notes):
            if current_price >= price:
                price_diff_percentage = (current_price - price) / price
                alerts.append(f'Price: {price}💰\nNote: {note}\nPrice Diff: {price_diff_percentage:.2%}\n')
    return alerts

# 定義檢查 Pullback 的函數
def check_pullback(current_price, last_price, pullback_prices, pullback_notes):
    alerts = ['拉回買']
    if pullback_prices is not None and pullback_notes is not None:
        for price, note in zip(pullback_prices, pullback_notes):
            if last_price >= price >= current_price:
                price_diff_percentage = (current_price - price) / price
                alerts.append(f'Price: {price}📈\nNote: {note}\nPrice Diff: {price_diff_percentage:.2%}\n')
    return alerts

# 定義檢查 Sell 的函數
def check_sell(current_price, last_price, sell_prices, sell_notes):
    alerts = ['停損']
    if sell_prices is not None and sell_notes is not None:
        for price, note in zip(sell_prices, sell_notes):
            if current_price <= price:
                price_diff_percentage = (current_price - price) / price
                alerts.append(f'Price: {price}🚨\nNote: {note}\nPrice Diff: {price_diff_percentage:.2%}\n')
    return alerts



In [32]:
notify_info = pd.read_excel('notify info.xlsx').astype(str)

In [34]:
text = 'ID       Status   Rank     Fund     Template'
for index, row in notify_info.iterrows():
    text += f'\n{row["stockID"]}  ||  {row["Status"]}  ||  {row["Rank"]}  ||  {row["Fundamental"]}  ||  {row["Template"]}'
print(text)


ID       Status   Rank     Fund     Template
3010  ||  持有  ||  A  ||  10/15  ||  T5
1477  ||  持有  ||  A  ||  10/15  ||  產+籌
2327  ||  持有  ||  B  ||  12/15  ||  產+籌
2317  ||  持有  ||  A  ||  12/15  ||  T6
3324  ||  買入  ||  B  ||  09/15  ||  產+籌


In [8]:
print(f'{bcolors.OK}reset   reset   reset   reset{bcolors.RESET}')
# 讀取 Excel 檔案
notify_info = pd.read_excel('notify info.xlsx').astype(str)

# 假設 stock_price_info 是最新的收盤價和成交量的 DataFrame
stock_price_info = pd.DataFrame({
    'close price': np.zeros(len(notify_info['stockID']), dtype=int),
    'volume': np.zeros(len(notify_info['stockID']), dtype=int)
}, index=notify_info['stockID'])

# 假設 last_stock_price_info 是上次的收盤價資訊
last_stock_price_info = pd.DataFrame({
    'close price': np.zeros(len(notify_info['stockID']), dtype=int),
    'volume': np.zeros(len(notify_info['stockID']), dtype=int)
}, notify_info['stockID'])
connection = 1
theurl = []
for id in notify_info['stockID']:
    # 去到你想要的網頁
    url = 'https://tw.stock.yahoo.com/quote/' + id.split("_")[0]
    theurl.append(url)
# for _ in range(10):
while True:
    try:
        posted = 0
        current_time = datetime.datetime.now()
        if not program_start :
            if current_time.hour < 9:
               # time.sleep(60)
                continue
            else:
                line_notify('\n【Program Start】', TOKEN_FOR_NOTIFY, notify_ornot)
                program_start = True
        if all([current_time.hour >= 13, current_time.minute >= 34, not program_test]):
            line_notify('\n【Program Shutdown】', TOKEN_FOR_NOTIFY, notify_ornot)
            sys.exit()
    ########################################################################################改頁數
        for num, id in enumerate(notify_info['stockID']):
            post = 0
            #儲存網址 
            T = np.random.randint(7,size = 1)[0]
            # 去到個股網頁
            try:
                try:
                    driver.get(theurl[num]+'.TW')
                    title = driver.find_element(by = By.ID, value = "qsp-overview-realtime-info").text
                    print(theurl[num]+'.TW')
                except:
                    driver.get(theurl[num]+'.TWO')
                    title = driver.find_element(by = By.ID, value = "qsp-overview-realtime-info").text
                    print(theurl[num]+'.TWO')
                if connection == 0:
                    line_notify('\n【Reconnect】', TOKEN_FOR_NOTIFY, notify_ornot)
                    print(f'{bcolors.OK}Reconnect{bcolors.RESET}')
                    connection = 1
                # time.sleep(T)
            except Exception as e:
                line_notify(f'\n【Disconnect : Request {id.split("_")[0]} Web Failed】', TOKEN_FOR_NOTIFY, notify_ornot)
                print(f'{bcolors.WARNING}Request Web Failed{bcolors.RESET}')
                print(e)
                print(f'{bcolors.WARNING}Disconnect{bcolors.RESET}')
                try:
                    driver.close()
                except Exception as e:
                    print(e)
                    print(f'{bcolors.WARNING}Close Google Failed{bcolors.RESET}')
                    pass
                #time.sleep(2)
                # service = ChromeService(executable_path=ChromeDriverManager().install())
                driver = webdriver.Chrome(service=service, options=chrome_options)
                connection = 0
                print('---------------------------------------------------------------------------------------------------------------------------------------------')
                continue
            #time.sleep(3)
            try:
                title = title.split('\n')
                # 
                name = title[0]
                # 日期
                data_time = title[1].split('：')[1]
                print(id)
                print(data_time)
                # 現在時間(小時)
                now_hour = data_time.split(' ')[1].split(':')[0]
                # 現在時間(分鐘)
                now_min = data_time.split(' ')[1].split(':')[1]
                # 現價
                price = title[4].replace(',','')
                stock_price_info.loc[id, 'close price'] = float(price)
                # 今日目前總量
                volume = title[22]
                stock_price_info.loc[id, 'volume'] = float(volume.replace(',',''))
                # 昨日總量
                yesterday_volume = title[24]
                last_stock_price_info.loc[id, 'volume'] = float(yesterday_volume.replace(',',''))
                # 昨日收盤
                yesterday_close = title[16].replace(',','')
                last_stock_price_info.loc[id, 'close price'] = float(yesterday_close)
                print(f"yesterday close:{yesterday_close}")
            except Exception as e:
                line_notify(f'\n【Error : Get {id.split("_")[0]} Data Failed】', TOKEN_FOR_NOTIFY, notify_ornot)
                print(f'{bcolors.FAIL}Get Data Failed{bcolors.RESET}')
                print(e)
                print('---------------------------------------------------------------------------------------------------------------------------------------------')
                continue
    except Exception as e:
        line_notify(f'\n【Error : Get {id.split("_")[0]} Data Failed】', TOKEN_FOR_NOTIFY, notify_ornot)
        print(f'{bcolors.FAIL}Program Failed{bcolors.RESET}')
        print(e)
        print('---------------------------------------------------------------------------------------------------------------------------------------------')
        continue
    is_notify = False
    for index, row in notify_info.iterrows():
        stock_id = row['stockID']
        # 準備所有提醒文本
        note_texts = ''
        # 將多個價位和 note 分割成列表，處理可能為 None 的情況
        breakout_prices = [float(x.replace(',', '')) for x in row['Breakout price'].split(',')] if row['Breakout price'] else []
        breakout_notes = row['Breakout note'].replace('\n','').split(',') if row['Breakout note'] else []
        profit_prices = [float(x.replace(',', '')) for x in row['Profit price'].split(',')] if row['Profit price'] else []
        profit_notes = row['Profit note'].replace('\n','').split(',') if row['Profit note'] else []
        pullback_prices = [float(x.replace(',', '')) for x in row['Pullback price'].split(',')] if row['Pullback price'] else []
        pullback_notes = row['Pullback note'].replace('\n','').split(',') if row['Pullback note'] else []
        sell_prices = [float(x.replace(',', '')) for x in row['Sell price'].split(',')] if row['Sell price'] else []
        sell_notes = row['Sell note'].replace('\n','').split(',') if row['Sell note'] else []
        
        current_price = stock_price_info.at[stock_id, 'close price']
        last_price = last_stock_price_info.at[stock_id, 'close price']
        est_volume = estimate_volume(int(now_hour), int(now_min), stock_price_info.at[stock_id, 'volume']).split('.')[0]
        avg50_volume = volume_avg(stock_id.split("_")[0]).split('.')[0]
        Template = row['Template']
        Rank = row['Rank']
        Fundamentals = row['Fundamental']
        shares = row['shares per 0.1position']

        
        # 檢查每個條件
        breakout_notes = check_breakout(current_price, last_price, breakout_prices, breakout_notes)
        profit_notes = check_profit(current_price, last_price, profit_prices, profit_notes)
        pullback_notes = check_pullback(current_price, last_price, pullback_prices, pullback_notes)
        sell_notes = check_sell(current_price, last_price, sell_prices, sell_notes)
        
        # 整合所有提醒文本
        notes = [breakout_notes, profit_notes, pullback_notes, sell_notes]
        # if notes:
            # note_texts.append(f"StockID {stock_id}:\n{notes}")
        for note in notes:
            if np.shape(note)[0] > 1:
                note_texts += '======================\n'
                for note_index, n in enumerate(note):
                    note_texts += f"{n}\n"
                    if note_index == 0:
                        note_texts += '======================\n'
                # note_texts += '===================\n'
        if note_texts:
            note_texts = f"\n【{stock_id}】\nTemplate: {Template}\nRank: {Rank}\nFundamentals: {Fundamentals}\nShares/0.1Position: {shares}\n\nLast price:{last_price}\nCurrent price:{current_price}\n\n{note_texts}"
            note_texts += f"目前成交量:{stock_price_info.at[stock_id, 'volume']}\n預估量:{est_volume}\n昨日總量:{last_stock_price_info.at[stock_id, 'volume']}\n50日均量:{avg50_volume}\n\n"
            line_notify(note_texts, TOKEN_FOR_NOTIFY, notify_ornot) #TOKEN_FOR_NOTIFY
            is_notify = True
        print(note_texts)
    if is_notify:
        sigment_line = '==== Sigment Line ===='
        line_notify(sigment_line, TOKEN_FOR_NOTIFY, notify_ornot)
    time.sleep(300)
    # 更新 last_stock_price_info 為當前的 stock_price_info
    # last_stock_price_info = stock_price_info.copy()

reset   reset   reset   reset
https://tw.stock.yahoo.com/quote/3010.TW
3010
2024/05/13 10:08
yesterday close:131.0
https://tw.stock.yahoo.com/quote/1477.TW
1477
2024/05/13 10:11
yesterday close:380.0
https://tw.stock.yahoo.com/quote/2327.TW
2327
2024/05/13 10:11
yesterday close:635
https://tw.stock.yahoo.com/quote/2317.TW
2317
2024/05/13 10:11
yesterday close:169.5
https://tw.stock.yahoo.com/quote/6526.TW
6526
2024/05/13 10:11
yesterday close:670





【6526】
Template: 產
Rank: B
Fundamentals: 10/15
Shares/0.1Position: 42

Last price:670.0
Current price:663.0

停損
Price: 669.0🚨
Note: 賣1
Price Diff: -0.90%

目前成交量:324
預估量:810
昨日總量:1361
50日均量:1334


https://tw.stock.yahoo.com/quote/3010.TW
3010
2024/05/13 10:16
yesterday close:131.0
https://tw.stock.yahoo.com/quote/1477.TW
1477
2024/05/13 10:12
yesterday close:380.0
https://tw.stock.yahoo.com/quote/2327.TW
2327
2024/05/13 10:15
yesterday close:635
https://tw.stock.yahoo.com/quote/2317.TW
2317
2024/05/13 10:16
yesterday close:169.5
https://tw

SystemExit: 